In [1]:
#import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gmaps

from config import census_key,g_key

gmaps.configure(g_key)

In [2]:
data = pd.read_csv("../Output/Data/census_data.csv")
byzipcode = data.copy()
races = byzipcode.columns[4:]
for race in races:
    byzipcode[f'{race}%'] = byzipcode[race]/byzipcode['population']

lat_log = pd.read_csv('../Resources/us-zip-code-latitude-and-longitude.csv', sep = ';')
lat_log = lat_log[['Zip','geopoint']]

byzipcode = pd.merge(lat_log,byzipcode, how = 'right', right_on = 'zipcode', left_on = 'Zip')
byzipcode.drop(columns= 'zipcode', inplace = True)
byzipcode.rename(columns = {'Zip':'zipcode'},inplace = True)
byzipcode.head()

,zipcode,geopoint,city,population,per_capita_income,white,african_american,native_american,asian,native_hawaiian_pacific_islander,...,hispanic_or_latino,white%,african_american%,native_american%,asian%,native_hawaiian_pacific_islander%,other_race%,two_or_more_races%,white_not_hispanic%,hispanic_or_latino%
0,46214,"39.792993,-86.28575",Indianapolis,24202.0,26462.0,12760.0,8746.0,30.0,446.0,189.0,...,3312.0,0.527229,0.361375,0.001240,0.018428,0.007809,0.035865,0.048054,0.450459,0.136848
1,75087,"32.93382,-96.45446",Dallas,33638.0,39436.0,30220.0,954.0,90.0,873.0,0.0,...,4076.0,0.898389,0.028361,0.002676,0.025953,0.000000,0.017926,0.026696,0.797342,0.121172
2,30336,"33.744007,-84.56032",Atlanta,234.0,42524.0,33.0,186.0,0.0,7.0,0.0,...,11.0,0.141026,0.794872,0.000000,0.029915,0.000000,0.000000,0.034188,0.123932,0.047009
3,19104,"39.961612,-75.19957",Philadelphia,54311.0,15478.0,20419.0,22962.0,291.0,7662.0,62.0,...,2766.0,0.375964,0.422787,0.005358,0.141076,0.001142,0.015430,0.038243,0.345823,0.050929
4,19146,"39.938512,-75.18067",Philadelphia,38873.0,45507.0,18549.0,16645.0,120.0,1701.0,35.0,...,1799.0,0.477169,0.428189,0.003087,0.043758,0.000900,0.015023,0.031873,0.446042,0.046279


In [3]:
# seattle_df = pd.read_csv("../Output/Data/seattle_restaurants.csv")
# phila_df = pd.read_csv("../Output/Data/philadelphia_restaurants.csv")
# dallas_df = pd.read_csv("../Output/Data/dallas_restaurants.csv")
# indie_df = pd.read_csv("../Output/Data/indianapolis_restaurants.csv")
# alt_df = pd.read_csv("../Output/Data/atlanta_restaurants.csv")

# yelp_df = pd.concat([seattle_df,phila_df,dallas_df,indie_df,alt_df])
# yelp_df = yelp_df.reset_index()
# yelp_df.drop(columns = ['index','Unnamed: 0'], inplace = True)
# yelp_df

In [11]:
yelp_df = pd.read_csv("../Output/Data/yelp_data.csv")
yelp_df

,name,city,latitude,longitude,zipcode,ethnicity,price,rating
0,Chubby Cattle,Philadelphia,39.954980,-75.156220,19107,chinese,NaN,4.5
1,Nan Zhou Hand Drawn Noodle House,Philadelphia,39.955401,-75.156901,19107,chinese,$,4.0
2,Dim Sum Garden,Philadelphia,39.955399,-75.156727,19107,chinese,$,4.0
3,Dim Sum House,Philadelphia,39.951885,-75.172466,19103,chinese,$$,4.0
4,Dan Dan,Philadelphia,39.950249,-75.167595,19102,chinese,$$,4.0
...,...,...,...,...,...,...,...,...
5739,Hanuman Thai Cafe,Seattle,47.676310,-122.207510,98033,thai,$$,3.5
5740,Jasmine Thai Cuisine,Seattle,47.609533,-122.341359,98101,thai,$,3.0
5741,Thai Ginger,Seattle,47.591233,-122.334065,98134,thai,$$,2.5
5742,Nibbana Thai Restaurant,Seattle,47.615154,-122.196783,98004,thai,$$,3.5


In [19]:
df = pd.merge(yelp_df,byzipcode, how = 'left',on = 'zipcode')
df.head()

,name,city_x,latitude,longitude,zipcode,ethnicity,price,rating,geopoint,city_y,...,hispanic_or_latino,white%,african_american%,native_american%,asian%,native_hawaiian_pacific_islander%,other_race%,two_or_more_races%,white_not_hispanic%,hispanic_or_latino%
0,Chubby Cattle,Philadelphia,39.954980,-75.156220,19107,chinese,NaN,4.5,"39.952112,-75.15853",Philadelphia,...,703.0,0.590318,0.081557,0.005330,0.271612,0.00000,0.009565,0.041618,0.554907,0.051329
1,Nan Zhou Hand Drawn Noodle House,Philadelphia,39.955401,-75.156901,19107,chinese,$,4.0,"39.952112,-75.15853",Philadelphia,...,703.0,0.590318,0.081557,0.005330,0.271612,0.00000,0.009565,0.041618,0.554907,0.051329
2,Dim Sum Garden,Philadelphia,39.955399,-75.156727,19107,chinese,$,4.0,"39.952112,-75.15853",Philadelphia,...,703.0,0.590318,0.081557,0.005330,0.271612,0.00000,0.009565,0.041618,0.554907,0.051329
3,Dim Sum House,Philadelphia,39.951885,-75.172466,19103,chinese,$$,4.0,"39.952162,-75.17406",Philadelphia,...,1377.0,0.773401,0.061894,0.002973,0.117263,0.00128,0.004253,0.038936,0.721252,0.056856
4,Dan Dan,Philadelphia,39.950249,-75.167595,19102,chinese,$$,4.0,"39.952962,-75.16558",Philadelphia,...,259.0,0.778205,0.044561,0.002228,0.163865,0.00000,0.002836,0.008305,0.725744,0.052461


In [14]:
#zipcodes where there is no census data or is not actually part of the city
df[df['city_y'].isnull()]['zipcode'].unique()

array([19096, 19082,  8103, 19004,  8102,  8108, 19003, 19050,  8030,
       19023, 19032, 19026,  8104,  8078,  8110,  8035, 19012,  8007,
        8045, 19072,  8033,  8031, 19018,  8002, 19027,  8109, 19195,
        8107,  8106, 19079, 19083, 19076,  8105, 19036, 19112,  8093,
       19029,  8059,  8100,  8086, 19041, 19155, 19176,  8063,  8096,
       75244, 75042, 75211, 75217, 75254, 75270, 46815, 75149, 75390,
       75001, 75044, 46203, 46250, 46227, 46240, 46077, 47906, 47901,
       47305, 46032, 47904, 46107, 46228, 73505, 46216, 46262, 46255,
       45245, 46206, 30329, 30033, 30341, 30340, 30360, 30339, 30345,
       30328, 30080, 30030, 30302, 30358, 30366, 30067, 30079, 30084,
       30060, 31634, 32011, 30322, 30036, 30144, 30361, 30101, 30034,
       98004, 98108, 98040, 98033, 98034, 98056, 98127, 98155, 98006,
       98195, 98005])

In [21]:
#drop these rows. can't use simple drop na as we want to keep NaA in price
df.drop(index = df[df['city_y'].isnull()].index, inplace = True)
df.drop(columns = ['city_y'], inplace = True)
df.rename(columns = {'name' : 'rest_name',
                     'latitude': 'rest_latitude',
                     'longitude' : 'rest_longitude',
                     'city_x' : 'city', 
                     'geopoint': 'zipcode geopoint', 
                     'ethnicity':'rest_ethnicity'}, inplace = True)

In [22]:
df.columns

Index(['rest_name', 'city', 'rest_latitude', 'rest_longitude', 'zipcode',
       'rest_ethnicity', 'price', 'rating', 'zipcode geopoint', 'population',
       'per_capita_income', 'white', 'african_american', 'native_american',
       'asian', 'native_hawaiian_pacific_islander', 'other_race',
       'two_or_more_races', 'white_not_hispanic', 'hispanic_or_latino',
       'white%', 'african_american%', 'native_american%', 'asian%',
       'native_hawaiian_pacific_islander%', 'other_race%',
       'two_or_more_races%', 'white_not_hispanic%', 'hispanic_or_latino%'],
      dtype='object')

In [23]:
#want to reorder columns
df = df[['rest_name','rest_ethnicity','rest_latitude', 'rest_longitude',
        'zipcode','zipcode geopoint','city','price', 'rating', 
       'population', 'per_capita_income', 'white', 'african_american',
       'native_american', 'asian', 'native_hawaiian_pacific_islander',
       'other_race', 'two_or_more_races', 'white_not_hispanic',
       'hispanic_or_latino', 'white%', 'african_american%', 'native_american%',
       'asian%', 'native_hawaiian_pacific_islander%', 'other_race%',
       'two_or_more_races%', 'white_not_hispanic%', 'hispanic_or_latino%']]

In [24]:
df

,rest_name,rest_ethnicity,rest_latitude,rest_longitude,zipcode,zipcode geopoint,city,price,rating,population,...,hispanic_or_latino,white%,african_american%,native_american%,asian%,native_hawaiian_pacific_islander%,other_race%,two_or_more_races%,white_not_hispanic%,hispanic_or_latino%
0,Chubby Cattle,chinese,39.954980,-75.156220,19107,"39.952112,-75.15853",Philadelphia,NaN,4.5,13696.0,...,703.0,0.590318,0.081557,0.005330,0.271612,0.000000,0.009565,0.041618,0.554907,0.051329
1,Nan Zhou Hand Drawn Noodle House,chinese,39.955401,-75.156901,19107,"39.952112,-75.15853",Philadelphia,$,4.0,13696.0,...,703.0,0.590318,0.081557,0.005330,0.271612,0.000000,0.009565,0.041618,0.554907,0.051329
2,Dim Sum Garden,chinese,39.955399,-75.156727,19107,"39.952112,-75.15853",Philadelphia,$,4.0,13696.0,...,703.0,0.590318,0.081557,0.005330,0.271612,0.000000,0.009565,0.041618,0.554907,0.051329
3,Dim Sum House,chinese,39.951885,-75.172466,19103,"39.952162,-75.17406",Philadelphia,$$,4.0,24219.0,...,1377.0,0.773401,0.061894,0.002973,0.117263,0.001280,0.004253,0.038936,0.721252,0.056856
4,Dan Dan,chinese,39.950249,-75.167595,19102,"39.952962,-75.16558",Philadelphia,$$,4.0,4937.0,...,259.0,0.778205,0.044561,0.002228,0.163865,0.000000,0.002836,0.008305,0.725744,0.052461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5731,Thai 2 Go,thai,47.604590,-122.333370,98104,"47.60252,-122.32855",Seattle,$,3.5,14144.0,...,1137.0,0.525099,0.103224,0.017958,0.250354,0.003323,0.029765,0.070277,0.486143,0.080387
5732,Thai Place,thai,47.597370,-122.327310,98104,"47.60252,-122.32855",Seattle,$,2.5,14144.0,...,1137.0,0.525099,0.103224,0.017958,0.250354,0.003323,0.029765,0.070277,0.486143,0.080387
5738,Basil Box,thai,47.604530,-122.330610,98104,"47.60252,-122.32855",Seattle,$,2.5,14144.0,...,1137.0,0.525099,0.103224,0.017958,0.250354,0.003323,0.029765,0.070277,0.486143,0.080387
5740,Jasmine Thai Cuisine,thai,47.609533,-122.341359,98101,"47.61067,-122.33438",Seattle,$,3.0,12792.0,...,609.0,0.680894,0.060507,0.012273,0.180972,0.003049,0.014462,0.047842,0.653924,0.047608


## Demographic Analysis 

## Pricing Analysis 

In [51]:
price_df = df.dropna()
price_df['price_int'] = [ len(price) for price in price_df['price']]


/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,rest_name,rest_ethnicity,rest_latitude,rest_longitude,zipcode,zipcode geopoint,city,price,rating,population,...,white%,african_american%,native_american%,asian%,native_hawaiian_pacific_islander%,other_race%,two_or_more_races%,white_not_hispanic%,hispanic_or_latino%,price_int
1,Nan Zhou Hand Drawn Noodle House,chinese,39.955401,-75.156901,19107,"39.952112,-75.15853",Philadelphia,$,4.0,13696.0,...,0.590318,0.081557,0.005330,0.271612,0.000000,0.009565,0.041618,0.554907,0.051329,1
2,Dim Sum Garden,chinese,39.955399,-75.156727,19107,"39.952112,-75.15853",Philadelphia,$,4.0,13696.0,...,0.590318,0.081557,0.005330,0.271612,0.000000,0.009565,0.041618,0.554907,0.051329,1
3,Dim Sum House,chinese,39.951885,-75.172466,19103,"39.952162,-75.17406",Philadelphia,$$,4.0,24219.0,...,0.773401,0.061894,0.002973,0.117263,0.001280,0.004253,0.038936,0.721252,0.056856,2
4,Dan Dan,chinese,39.950249,-75.167595,19102,"39.952962,-75.16558",Philadelphia,$$,4.0,4937.0,...,0.778205,0.044561,0.002228,0.163865,0.000000,0.002836,0.008305,0.725744,0.052461,2
5,Sampan,chinese,39.949531,-75.162157,19107,"39.952112,-75.15853",Philadelphia,$$,4.0,13696.0,...,0.590318,0.081557,0.005330,0.271612,0.000000,0.009565,0.041618,0.554907,0.051329,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5731,Thai 2 Go,thai,47.604590,-122.333370,98104,"47.60252,-122.32855",Seattle,$,3.5,14144.0,...,0.525099,0.103224,0.017958,0.250354,0.003323,0.029765,0.070277,0.486143,0.080387,1
5732,Thai Place,thai,47.597370,-122.327310,98104,"47.60252,-122.32855",Seattle,$,2.5,14144.0,...,0.525099,0.103224,0.017958,0.250354,0.003323,0.029765,0.070277,0.486143,0.080387,1
5738,Basil Box,thai,47.604530,-122.330610,98104,"47.60252,-122.32855",Seattle,$,2.5,14144.0,...,0.525099,0.103224,0.017958,0.250354,0.003323,0.029765,0.070277,0.486143,0.080387,1
5740,Jasmine Thai Cuisine,thai,47.609533,-122.341359,98101,"47.61067,-122.33438",Seattle,$,3.0,12792.0,...,0.680894,0.060507,0.012273,0.180972,0.003049,0.014462,0.047842,0.653924,0.047608,1
